In [1]:
from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di # Example: di.display_html('<h3>%s:</h3>' % str, raw=True)

display(HTML("<style>.container { width:100% !important; }</style>"))


import hail as hl
import sys
import os

hl.init(tmp_dir='temp', spark_conf={'spark.driver.memory': '20G', 'spark.executor.memory': '20G'}, default_reference='GRCh38') 

2022-01-18 12:22:32 WARN  NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Running on Apache Spark version 3.1.2
SparkUI available at http://p1517.prometheus:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.79-f141af259254
LOGGING: writing to /net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/preprocessing/joint-with-gts/hail-20220118-1222-0.2.79-f141af259254.log


In [ ]:
from hail.plot import show
from pprint import pprint
from bokeh.layouts import gridplot
hl.plot.output_notebook()
import numpy as np
import pandas as pd
from functools import reduce
from itertools import chain
from bokeh.plotting import output_notebook, show, figure
from bokeh.palettes import viridis

output_notebook() 

## genotype all:

In [ ]:
sparse_sport = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/joint/sport-sparse.mt')
sparse_1kg = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/external-data/1kg/1kg-europeans-sparse.ht/')
sparse_gts = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/joint-gts-only/gts-sparse.mt/')

sparse_1kg = sparse_1kg.key_rows_by(sparse_1kg.locus)
sparse_gts = sparse_gts.key_rows_by(sparse_gts.locus)
sparse_sport = sparse_sport.key_rows_by(sparse_sport.locus)

sparse_sport = sparse_sport.drop(sparse_sport.gvcf_info)
sparse_1kg = sparse_1kg.drop(sparse_1kg.gvcf_info)            
sparse_gts = sparse_gts.drop(sparse_gts.gvcf_info)

sparse_sport = sparse_sport.select_entries('DP', 'END', 'GQ', 'LA',
                                           'LAD', 'LGT', 'LPGT', 'LPL',
                                           'MIN_DP', 'PID', 'RGQ', 'SB')

sparse_gts = sparse_gts.select_entries('DP', 'END', 'GQ', 'LA',
                                           'LAD', 'LGT', 'LPGT', 'LPL',
                                           'MIN_DP', 'PID', 'RGQ', 'SB')

sparse_gts.naive_coalesce(20000).write('gts-sparse')
sparse_sport.naive_coalesce(20000).write('sport-sparse')
sparse_1kg.naive_coalesce(20000).write('1kg-sparse')

In [ ]:
sparse_gts = hl.read_matrix_table('gts-sparse')
sparse_sport = hl.read_matrix_table('sport-sparse')
sparse_1kg = hl.read_matrix_table('1kg-sparse')

In [ ]:
sparse_gts.naive_coalesce(5000).write('gts-sparse-repart')
sparse_sport.naive_coalesce(5000).write('sport-sparse-repart')
sparse_1kg.naive_coalesce(5000).write('1kg-sparse-repart')

In [ ]:
sparse_gts = hl.read_matrix_table('gts-sparse-repart')
sparse_sport = hl.read_matrix_table('sport-sparse-repart')
sparse_1kg = hl.read_matrix_table('1kg-sparse-repart')

In [ ]:
sparse_1 = hl.experimental.vcf_combiner.vcf_combiner.combine_gvcfs([sparse_sport,sparse_gts])
sparse_1.write('to-delete-01')
sprarse_1 = hl.read_matrix_table('to-delete-01')
sparse_all = hl.experimental.vcf_combiner.vcf_combiner.combine_gvcfs([sparse_1,sparse_1kg])
sparse_all.write('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/joint/sparse_all.mt')

In [ ]:
sparse = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/joint/sparse_all.mt')

In [ ]:
dense = hl.experimental.densify(sparse)
dense = dense.annotate_entries(GT=hl.experimental.lgt_to_gt(dense.LGT, dense.LA))
dense = dense.filter_rows(hl.agg.any(dense.GT.is_non_ref()))
dense.write('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/joint/dense-all.mt')

In [ ]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/joint/dense-all.mt')

samples_to_keep = hl.import_table('/net/archive/groups/plggneuromol/matzieb/projects/imdik-zekanowski-sportwgs/data/prs-data/sportsmen-control-pheno.tsv')
samples_to_keep = samples_to_keep.key_by(samples_to_keep['sample'])
sample_filter = hl.literal(samples_to_keep['sample'].collect()).contains(mt.s)

mt = mt.filter_cols(hl.if_else(mt.s.contains('B'), sample_filter,
                          hl.if_else(mt.s.contains('HG'), sample_filter, hl.if_else(
           mt.s.contains('NA'), sample_filter, True))))

mt = mt.annotate_cols(group = hl.if_else(
     mt.s.contains('B'), 'sport', hl.if_else(
     mt.s.contains('HG'), '1kg', hl.if_else(
           mt.s.contains('NA'), '1kg', 'GTS'))))

mt = mt.filter_rows(hl.agg.any(mt.GT.is_non_ref()))

rpmk = hl.read_table('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/external-data/repeatmasker-extended-keyed.ht')
cov = hl.read_table('/net/archive/groups/plggneuromol/ifpan-gosborcz-ukb/raw/gnomad/gnomad.genomes.r3.0.1.coverage.ht')

cov = cov.filter(cov.over_1 > 0.9)

In [ ]:
mt = mt.filter_rows(hl.is_defined(rpmk[mt.locus]), keep = False)
mt = mt.filter_rows(hl.is_defined(cov[mt.locus]), keep = True)

mt = mt.annotate_rows(dp_qc = hl.agg.group_by(mt.group, hl.agg.stats(mt.DP)),
                      gq_qc = hl.agg.group_by(mt.group, hl.agg.stats(mt.GQ)),
                      hwe = hl.agg.group_by(mt.group, hl.agg.hardy_weinberg_test(mt.GT)))

mt = mt.annotate_rows(n_below_dp_3 = hl.agg.group_by(mt.group, hl.agg.count_where(mt.DP < 3)),
                      n_below_gq_30 = hl.agg.group_by(mt.group, hl.agg.count_where(mt.GQ <30)))
    
mt = mt.filter_rows((mt.dp_qc.get('GTS', hl.struct(mean=0, stdev=0.0, min=0.0, max=0.0, n=0, sum=0.0)).mean > 5) &
                    (mt.dp_qc.get('1kg', hl.struct(mean=0, stdev=0.0, min=0.0, max=0.0, n=0, sum=0.0)).mean > 5) &
                    (mt.dp_qc.get('sport', hl.struct(mean=0, stdev=0.0, min=0.0, max=0.0, n=0, sum=0.0)).mean > 5) &
                    (mt.gq_qc.get('GTS', hl.struct(mean=0.0, stdev=0.0, min=0.0, max=0.0, n=0, sum=0.0)).mean > 50) &
                    (mt.gq_qc.get('1kg', hl.struct(mean=0.0, stdev=0.0, min=0.0, max=0.0, n=0, sum=0.0)).mean > 50) &
                    (mt.gq_qc.get('sport', hl.struct(mean=0.0, stdev=0.0, min=0.0, max=0.0, n=0, sum=0.0)).mean > 50) &
                    (mt.hwe.get('GTS', hl.struct(het_freq_hwe=0.0, p_value=0.5)).p_value > 0.05) &
                    (mt.hwe.get('sport', hl.struct(het_freq_hwe=0.0, p_value=0.5)).p_value > 0.05) &
                    (mt.hwe.get('1kg', hl.struct(het_freq_hwe=0.0, p_value=0.5)).p_value > 0.05) &
                    (mt.n_below_dp_3.get('sport', 0) < 3) &
                    (mt.n_below_gq_30.get('sport', 0) < 30) &
                    (mt.n_below_dp_3.get('1kg', 0) < 3) &
                    (mt.n_below_gq_30.get('1kg', 0) < 30) &
                    (mt.n_below_dp_3.get('GTS', 0) < 3) &
                    (mt.n_below_gq_30.get('GTS', 0) <30))

In [ ]:
mt.write('filtered.mt')

In [ ]:
mt = hl.read_matrix_table('filtered.mt')

In [ ]:
genes = hl.read_table('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/external-data/genecode_v32.ht')
genes = genes.filter(hl.is_valid_contig(genes['hg38.knownGene.chrom'], reference_genome='GRCh38'))
hpo = hl.import_table('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/external-data/hpo.tsv', impute = True, no_header=True)

start = genes['hg38.knownGene.txStart']
stop =  genes['hg38.knownGene.txEnd']

genes = genes.transmute(interval = 
                        hl.locus_interval(genes['hg38.knownGene.chrom'], 
                                          start,
                                          stop,
                                          reference_genome='GRCh38', includes_start=False))

genes = genes.key_by(genes['hg38.kgXref.geneSymbol'])

hpo = hpo.key_by(hpo.f0)

genes = genes.annotate(hpo = hpo.index(genes['hg38.kgXref.geneSymbol'], all_matches = True)['f1'])
genes = genes.key_by(genes.interval)

gnomad = hl.read_table('/net/archive/groups/plggneuromol/ifpan-gosborcz-ukb/raw/gnomad/gnomad.genomes.v3.1.1.sites.ht/')
cadd = hl.read_table('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/external-data/cadd-full.ht')
vep = hl.read_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/external-data/vep38/grch38_context_vep_annotated.ht')
sport_pheno = hl.import_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/external-data/sportsmen-pheno.csv', impute = True, key='sample_id', quote ="\"")
poles = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/external-data/polish-genomes/polish-genomes.mt')

mt = hl.read_matrix_table('filtered.mt')
mt = mt.distinct_by_row()
mt = mt.key_rows_by(mt.locus, mt.alleles)
mt = hl.split_multi_hts(mt)
mt = mt.annotate_rows(gnomad_v_3_1 = gnomad[mt.row_key])
mt = mt.annotate_rows(within_gene = hl.array(hl.set(genes.index(mt.locus, all_matches=True)['hg38.kgXref.geneSymbol'])))
mt = mt.annotate_rows(hpo = hl.array(hl.set(genes.index(mt.locus, all_matches=True)['hpo'])))
mt = mt.annotate_rows(cadd = cadd[mt.row_key])
mt = mt.annotate_rows(vep = vep[mt.row_key])
mt = mt.annotate_cols(sport_phenotypes = sport_pheno[mt.s])
mt = mt.annotate_rows(polish_af = poles.rows()[mt.row_key]['info'])

mt = mt.annotate_cols(group = hl.if_else(
     mt.s.contains('B'), 'sport', hl.if_else(
     mt.s.contains('HG'), '1kg', hl.if_else(
           mt.s.contains('NA'), '1kg', 'control'))))

mt = mt.annotate_rows(het_non_refs = hl.agg.group_by(mt.group, hl.agg.count_where(mt.GT.is_het())))
mt = mt.annotate_rows(hom_refs = hl.agg.group_by(mt.group, hl.agg.count_where(mt.GT.is_hom_ref())))
mt = mt.annotate_rows(hom_non_refs = hl.agg.group_by(mt.group, hl.agg.count_where(mt.GT.is_hom_var())))

mt = mt.annotate_rows(het_non_refs_e_vs_s = hl.agg.group_by(mt.sport_phenotypes.type, hl.agg.count_where(mt.GT.is_het())))
mt = mt.annotate_rows(hom_refs_e_vs_s = hl.agg.group_by(mt.sport_phenotypes.type, hl.agg.count_where(mt.GT.is_hom_ref())))
mt = mt.annotate_rows(hom_non_refs_e_vs_s = hl.agg.group_by(mt.sport_phenotypes.type, hl.agg.count_where(mt.GT.is_hom_var())))

mt = mt.filter_cols(mt.s != '464')

mt.write('temp-anno.mt')

### Perform PCA to check:

In [ ]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/preprocessing/joint-with-gts/temp-anno.mt')

subset = mt.sample_rows(0.005)
subset.write('temp-for-pca.mt')

In [ ]:
subset = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/preprocessing/joint-with-gts/temp-for-pca.mt')

eigenvalues, pcs, _ = hl.hwe_normalized_pca(subset.GT)
mt = mt.annotate_cols(scores = pcs[mt.s].scores)
mt.write('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/joint/full.mt')

In [ ]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/joint/full.mt')

mt.aggregate_cols(hl.agg.counter(mt.group))

p = hl.plot.scatter(mt.scores[0],
                    mt.scores[1],
                    label=mt.group,
                    title='PCA', xlabel='PC1', ylabel='PC2')

show(p) #this shows families mainly so we are removing related individuals

### Selecting the matrixtable to contain only unrelated healthy controls from GTS:

In [ ]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/joint/full.mt')

In [ ]:
pheno = hl.import_table('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/pheno/GTS-coded-corrected-june-2021.csv', impute = True, key='ID', delimiter = ',', quote ="\"")
mt = mt.annotate_cols(gts_phenotypes = pheno[mt.s])

In [ ]:
#first of all remove individuals with GTS:

mt = mt.filter_cols(hl.if_else(hl.is_defined(mt.gts_phenotypes.disease), (mt.gts_phenotypes.disease != 'YES'), True))

In [ ]:
mt.write('temp-subset.mt')

In [ ]:
mt = hl.read_matrix_table('temp-subset.mt')

In [ ]:
#to remove - based on kinship analisys and PCA + one outlier sportsmen
samples_to_remove = ["475", "477", "478", "479", "483", "S_7214", "S_7227", "S_7229", "S_7247", "S_7291", "WGS_180b", "WGS_6819", "WGS_163b", "WGS_163a", "WGS_D6813", "WGS_6814", "S_7237"]

In [ ]:
mt = mt.filter_cols(
     hl.literal(samples_to_remove).contains(mt.s), keep = False)

In [ ]:
mt = mt.filter_rows(hl.agg.any(mt.GT.is_non_ref()))

In [ ]:
len(gts_controls)

In [ ]:
subset = mt.sample_rows(0.02)

In [ ]:
subset.write('temp-for-pca-12.mt')

In [ ]:
subset = hl.read_matrix_table('temp-for-pca-12.mt')

In [ ]:
eigenvalues, pcs, _ = hl.hwe_normalized_pca(subset.GT)

In [ ]:
mt = mt.annotate_cols(scores = pcs[mt.s].scores)

In [ ]:
p = hl.plot.scatter(mt.scores[0],
                    mt.scores[1],
                    label=mt.group,
                    title='PCA', xlabel='PC1', ylabel='PC2')

In [ ]:
show(p)

In [ ]:
mt.write('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/joint/full_healthy.mt')